# Labrapport som Jupyter Notebook på fysikklab våren 2023
Skrevet 16.03.2023 <br>
Skrevet av Emma Amalie Chun Hui Birkhaug Pedersen, Halvor Linder Henriksen, Oskar Jørgensen og Sindre Eidskrem <br>
Institutt for Fysikk, NTNU
***

## Sammendrag
Sammendraget inneholder en konsis oppsummering av innholdet i oppgaven. Det kan være en god idé å skrive denne delen til slutt, når du har god oversikt over hele teksten. Typisk inneholder sammendraget 4-5 setninger. På disse få setningene skal førsøket introduseres, de eksperimentelle og numeriske metodene nevnes, og resultatene - og deres betydning - skal presenteres. Sentrale tallverdier oppgis med tilhørende usikkerhet.

***

## Introduksjon
Selve rapporten starter her. Introduksjonen bør inneholde en tydelig motivasjon og hensikt ved forsøket. Her kan også teksten settes i en historisk sammenheng. Det vil i så fall være natulig å referere til relevant litteratur.
***

## Teori
Her presenteres relevant teori nødvendig for å lese resten av rapporten. Jupyter notebook støtter LaTeX-kode, så dere kan inkludere fint formaterte ligninger her. For eksempel har vi sett at baneakselerasjonen til ei kompakt kule som ruller på et krumt underlag uten å slure (gli) er gitt ved
$$a = -\frac{5g\sin{\beta}}{7}$$

der $g$ er tyngdens akselerasjon og $\beta$ er banens helningsvinkel. Her forklarer vi alle nye størrelser som blir innført med ord. <br>
For at dette skal ha god flyt, skal ligningene kunne leses som en naturlig del av setningen, slik som ovenfor. 

Teoridelen inneholder ikke referanser til selve forsøket. Den er generell og kan leses isolert fra resten av rapporten.

Her vil det ofte være naturlig å inkludere en eller flere forklarende figurer. Disse skal dere lage selv, for eksempel ved hjelp av [Inkscape](https://inkscape.org/) som er et nokså brukervennlig program for å lage vektorgrafikk. Her er et eksempel på en slik figur:


Under er det som vi har skrevet. Beholder dette ovenfor fordi det står litt om hvordan det skal skrives



### Ren rulling på krumt underlag
Her diskuteres teori rundt forsøket. Bevarelse av mekanisk energi og Newtons 2. lov brukes til å utlede likninger for hastighet, akselerasjon, krefter og energi.

#### Energi og Hastighet ved bevarelse av mekanisk energi
Siden det antas ren rulling (ingen sluring) er den mekaniske energien til kule bevart gjennom forsøket. Det vil si at summen av den potensille og kinetiske energien til kula vil være konstant gjennom forsøket.

$$E=U_i + K_i = U_j + K_j\tag{1}$$

Der henholdsvis $U_i$ og  $K_i$ er kulas potensielle og kinetiske energi ved punkt $i$. I starten av forsøket står kula stille og vi måler potensiell energi med høyde null som utgangspunkt. Da er den mekaniske energien i ved startpunktet lik denne potensielle energien gitt ved at kule befinner seg på høyden til startpunktet.

$$E = U_0 = mgy_0\tag{2}$$

Der $m$ er kulas masse, $g$ er tyngdens akselerasjon og $y_0$ er høyden til den første skruen (høyden til kula over nullpunktet).
Kulas kinetiske energi er ved etthvert punkt gitt som summen translasjonsenergien og rotasjonsenergien.

$$K =  \frac{mv^2}{2} + \frac{cmv^2}{2} = \frac{1+c}{2}mv^2 \tag{3}$$

Der $v$ er kulas hastighet og $c$ er konstansten som inngår i uttrykket for treghetsmomentet til rullende legemer. I dette tilfellet er legemet en kule og $c = 2/5$.

Ved å kombinere ligningene $(1), (2)$ og $(3)$ kan vi finne et utrykk for kulas hastighet gitt ved kulas vertikale posisjon $y$ (høyden til banen)

$$v(y) = \sqrt{\frac{2g(y_0 - y)}{1 + c}}  \tag{4}$$

Hastigheten kan også oppfattes som en funksjon av kulas horisontale posisjon $x$ siden vi kjenner baneformen som en funksjon $y(x)$.

#### Akselerasjon

Som nevnt kjenner vi baneformen $y(x)$. Vi kjenner også 1. og 2. deriverte av denne. Disse vil inngå i utrykkene for kulas akselerasjon.
Krumningsradien til  banen er gitt ved utrykket

$$ R = \frac{[1 + (y')^2]^{3/2}}{y''}  \tag{5}$$

Der $y''$ er den 2. deriverte av baneformen

Kulas sentripetalakselerasjon er gitt som 

$$a_\perp = \frac{v^2}{R} = \frac{2g(y_0 - y)}{1+c} * \frac{y''}{[1 + (y')^2]^{3/2}} \tag{6}$$

Fortegnet for $a_\perp$ vil være positivt når banen krummer oppover, og negativt når banen krummer nedover.

#### Krefter og Newtons 2. lov
Kreftene som virker på kula er tygndegraften $mg$, normalkraften fra banen på kula  $N$, og den statiske friksjonskraften $f$ mellom kula og banen.

For å finne normalkraften $N$, brukes Netwons 2. lov
 



In [1]:
from IPython import display
#display.Image("./krefter.png", width=300)

<b>Figur 1.</b> Kule som ruller på et krumt underlag. Her er $v$ kulas fart,
$a$ er baneakselerasjon, $a_\perp$ er sentripetalakselerasjon, $Mg$ er tyngdekraft,
$f$ er friksjonskraft og $N$ er normalkraft. <br><br>

***

## Metode
Dette avsnittet inneholder en forholdsvis detaljert beskrivelse av utstyr og metoder som er brukt i
prosjektet, både numeriske og eksperimentelle metoder.
En figur som viser det eksperimentelle oppsettet inngår naturlig her.
Metodedelen kan godt inneholde små kodesnutter som illustrerer hvordan ulike størrelser beregnes.
Disse vil da typisk inkluderes i Markdown-celler, slik at notebooken ikke prøver å kjøre dem.

Et konkret eksempel:

Baneformen fastlegges ved hjelp av tredjegradspolynomer på intervallene mellom de 8 tilfeldig
valgte skruefestehøydene, slik at både banen, dens deriverte og krumningen er kontinuerlige
i skruefestene. Dette gjøres i Python med funksjonen CubicSpline fra SciPy-biblioteket interpolate
med kommandoen

*cs = CubicSpline(xfast,yfast,bc_type='natural')*

Her er xfast og yfast tabeller (numpy arrays) med henholdsvis horisontale og vertikale koordinater 
for skruefestene, og med *bc_type='natural'* velges krumning lik null i de to ytterste skruefestene.


Deretter vil den konkrete <i>bruken</i> av funksjonen *CubicSpline* komme
i Resultat-avsnittet.
***

## Resultater
Her presenteres numeriske og eksperimentelle resultater. Dere inkluderer figurer i henhold til rapportkravene, samt tallverdier for kulas rulletid, slutthastighet, tilhørende kinetisk energi, og tap i mekanisk energi, med tilhørende usikkerhet. Anvendt pythonkode inngår naturlig her. En må forvente at dette avsnittet opptar størst plass i rapporten. 
***

## Diskusjon
Her diskuteres resultatene, både de numeriske og de eksperimentelle. <br>

- Kan forskjellene mellom numerisk og eksperimentell hastighet $v(x)$ forklares med utgangspunkt i
forskjellene mellom numerisk og eksperimentell baneform $y(x)$?
- Er grafene for $v(x)$, $N(x)$ og $f(x)$ rimelige med utgangspunkt i baneformen $y(x)$? 
Vurder spesielt topp-punkt, bunnpunkt(er) og vendepunkter.
- Er det sannsynlig at baneformen gir ren rulling? Dette avhenger av at beregnet friksjonskraft hele veien er mindre enn maksimal statisk friksjonskraft, som er gitt ved $\mu_s N$. Grafen for $|f(x)/N(x)|$ gir grunnlag for å diskutere dette.
- Er målt rulletid rimelig, med utgangspunkt i beregnet rulletid?
- Er målt tap i mekanisk energi rimelig? Diskuter feilkilder.
***

## Konklusjon
Dette er en kort oppsummering av det som ble gjort og hvilke resultater som ble oppnådd. Tallverdier for numerisk vs eksperimentell rulletid og sluttfart oppgis, samt tap i mekanisk energi med usikkerhet. 

## Referanser
En labrapport som denne vil typisk inneholde et *lite* antall referanser. Av den grunn
er det greit å nummerere disse manuelt. Hvis du sjekker ulike
tidsskrifter, vil du se at ulike konvensjoner for formatering av referanser benyttes.
Det viktige er at referanselista formateres systematisk. Nedenfor viser vi eksempler på
henvisning til tidsskriftartikler, bok og url, med formatering
som benyttet i tidsskriftet Physical Review Letters: <br>

[1] D. Xiao, W. Yao, and Q. Niu, Valley-Contrasting Physics in Graphene: Magnetic Moment and Topological Transport, Phys. Rev. Lett. <b>99</b>, 236809 (2007).  <br>

[2] D. Xiao, M.-C. Chang, and Q. Niu, Berry phase effects on electronic properties, 
Rev. Mod. Phys. <b>82</b>, 1959 (2010).  <br>

[3] T. Ihn, <i>Semiconductor Nanostructures: Quantum States and
Electronic Transport</i> (Oxford University Press, New York,
2010). <br>

[4] See Supplemental Materials at
<a href="http://link.aps.org/supplemental/10.1103/PhysRevLett.128.067702">
link.aps.org/supplemental/10.1103/PhysRevLett.128.067702</a>.
<br><br>

Til sammenligning, et par referanser fra en artikkel i Chemical Reviews: <br>

(1) Ayyagari, A. V.; Gwalani, B.; Muskeri, S.; Mukherjee, S.;
Banerjee, R. Surface degradation mechanisms in precipitationhardened high-entropy alloys. *npj Materials Degradation* <b>2018</b>, *2*, 33.  <br>
(2) Frankel, G. S.; Vienna, J. D.; Lian, J.; Scully, J. R.; Gin, S.; Ryan,
J. V.; Wang, J.; Kim, S. H.; Windl, W.; Du, J. A comparative review of
the aqueous corrosion of glasses, crystalline ceramics, and metals. *npj
Materials Degradation* <b>2018</b>, *2*, 15. <br>

Små, men *mange* forskjeller: type parentes i referansenummeret; plassering, parentesbruk og 
font for årstall; plassering av forbokstav i fornavn; *and* eller ikke *and* foran sistnevnte 
av tre eller flere forfattere. Ubetydelige valg i seg selv, men viktig at alt gjøres *systematisk*.
